In [1]:
import pandas as pd

import sys
sys.path.insert(1, '../../scripts/')
from s3_support import *

In [2]:
start_date = "2021-04-01"

# pulling page view syslog logs data

In [3]:
q = "select * from syslog_logs where created>'{}' and message LIKE '%Page view%'".format(start_date)
df = redshift_query_read(q, schema='production')

In [4]:
print("{:,}".format(len(df)))
print("{:,}".format(len(df[df['org']!=0])))

36,005,794
16,357,852


In [5]:
df[df['org']!=0][['org', 'form', 'type', 'message']].head()

,org,form,type,message
3,445458,956938,3,Donation dedication email triggered
26,29738,833630,2,Failed to send receipt notification to
46,446694,446694,3,added donorperfect standard Peer-to-Peer Cate...
56,437303,966807,3,Donation dedication email triggered
58,445882,969495,3,QR Code Creation Success


# exploring page views

In [6]:
cols = ['org', 'form', 'message']
df[df['message'].str.contains('Page view').fillna(False)][cols].tail()

,org,form,message
33653443,444339,444339,Page view: donors/search/
33653444,442558,442558,Page view: recent_activity/
33653445,442214,930716,Page view: forms/930716/events/828966/attendee...
33653632,447300,447300,Page view: signup/documents/
33653646,447300,447300,Page view: recent_activity/


In [7]:
print("{:,}".format(len(df[df['message'].str.contains('Page view').fillna(False)])))

16,408,802


In [8]:
df[df['message'].str.contains('Page view').fillna(False)]['message'].value_counts().head(20).reset_index()

,index,message
0,Page view: recent_activity/,3340714
1,Page view: login/,2368423
2,Page view: chart_new/,2163587
3,Page view: reports/transaction/search/,875610
4,Page view:,565468
5,Page view: login,463925
6,Page view: forms/,417008
7,Page view: reports/transaction/,229877
8,Page view: vt/,210980
9,Page view: reports/recurring/search/,120936


In [9]:
top_pages = df[df['message'].str.contains('Page view').fillna(False)]['message'].value_counts().head(50)
top_pages = top_pages.reset_index()
top_pages.columns = ['URL', 'Views']
top_pages = top_pages['URL'].tolist()
top_pages = [p.replace('Page view: ', '') for p in top_pages]

In [10]:
for page in top_pages:
    these_logs = df[df['message'].str.contains(page).fillna(False)]
    len_orgs = len(these_logs['org'].unique())
    mean_visits = these_logs.groupby('org')['id'].count().mean()
    
    print("{}: {:,} orgs, {:.2f} average visits per org".format(page, len_orgs, mean_visits))

recent_activity/: 3,564 orgs, 937.44 average visits per org
login/: 1,950 orgs, 1215.82 average visits per org
chart_new/: 3,562 orgs, 607.41 average visits per org
reports/transaction/search/: 2,577 orgs, 339.78 average visits per org
: 4,654 orgs, 7736.53 average visits per org
login: 2,070 orgs, 1369.58 average visits per org
forms/: 2,672 orgs, 1688.94 average visits per org
reports/transaction/: 2,577 orgs, 446.37 average visits per org
vt/: 1,297 orgs, 440.67 average visits per org
reports/recurring/search/: 1,110 orgs, 108.95 average visits per org
donors/search/: 1,608 orgs, 53.31 average visits per org
statements/: 2,065 orgs, 74.74 average visits per org
reports/: 2,783 orgs, 620.58 average visits per org
statements/settlements/: 1,071 orgs, 65.21 average visits per org
reports/recurring/: 1,113 orgs, 160.36 average visits per org
users/: 2,208 orgs, 37.45 average visits per org
reports/exports/: 910 orgs, 103.22 average visits per org
reports/transaction/download/: 1,216 org

In [11]:
df['message'].tail()

36005789    Queue Job #9639591 has been added into the queue
36005790    Queue Job #9639575 has been added into the queue
36005791    Queue Job #9639559 has been added into the queue
36005792    Queue Job #9639543 has been added into the queue
36005793                    Queue Job #9639530 has completed
Name: message, dtype: object

In [12]:
df[df['message'].str.contains(': recurring').fillna(False)]

,id,org,form,entity,entitytype,systemid,systemtype,type,created,userid,ghost,hidden,access,ack,count,message
161551,42043711,29769,29769,29769,0,0,0,3,2021-07-06 15:33:07,1027244,0,0,1,0,1,Page view: recurring/980416/
161786,42103577,436266,436266,436266,0,0,0,3,2021-07-07 10:00:26,1030602,0,0,1,0,1,Page view: recurring/1171217/billing/edit/
161893,42131410,443689,443689,443689,0,0,0,3,2021-07-07 12:37:57,1251254,0,0,1,0,1,Page view: recurring/1117435/
162242,42220420,436268,436268,436268,0,0,0,3,2021-07-08 09:11:15,1496518,0,0,1,0,1,Page view: recurring/1127770/
162359,42251149,441687,441687,441687,0,0,0,3,2021-07-08 12:27:01,1223284,0,0,1,0,1,Page view: recurring/1057061/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33652916,42032201,443101,443101,443101,0,0,0,3,2021-07-06 14:25:09,1228559,0,0,1,0,1,Page view: recurring/1076365
33653076,42029641,93187,93187,93187,0,0,0,3,2021-07-06 14:13:35,1449597,0,0,1,0,1,Page view: recurring/1149389/
33653145,42028537,444339,444339,444339,0,0,0,3,2021-07-06 14:07:31,1362865,0,0,1,0,1,Page view: recurring/1162914/
33653179,42027993,444339,444339,444339,0,0,0,3,2021-07-06 14:03:55,1362865,0,0,1,0,1,Page view: recurring/1157817/
